# 📊 Excel Analysis Report

**File:** `Business Accounting.xlsx`
**Analysis Date:** 2025-08-04 20:46:47
**Processing Time:** 0.69s

In [1]:
import pandas as pd
from pathlib import Path

# Load the Excel file (path relative to repo root)
excel_path = Path("test_assets/collection/business-accounting/Business Accounting.xlsx")
sheet_index = 2

print(f"Loading data from: {excel_path}")
print(f"Loading sheet at index: {sheet_index}")

try:
    # First, get information about all sheets
    xl_file = pd.ExcelFile(excel_path)
    sheet_names = xl_file.sheet_names
    print(f"\nAvailable sheets: {sheet_names}")

    if sheet_index >= len(sheet_names):
        print(f"\nError: Sheet index {sheet_index} is out of range. File has {len(sheet_names)} sheets.")
        print("Please use --sheet-index with a value between 0 and {}")
        df = None
    else:
        # Load the specified sheet
        selected_sheet = sheet_names[sheet_index]
        print(f"\nLoading sheet: '{selected_sheet}'")

        df = pd.read_excel(excel_path, sheet_name=sheet_index)
        print(f"\nLoaded data with shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")

        # Display first few rows
        print("\nFirst 5 rows:")
        display(df.head())

        # Basic info
        print("\nData types:")
        print(df.dtypes)

except Exception as e:
    print(f"Error loading Excel file: {e}")
    df = None


Loading data from: test_assets/collection/business-accounting/Business Accounting.xlsx
Loading sheet at index: 2

Available sheets: ['Yiriden Transactions 2025', 'Yiriden Transactions 2023', 'Yiriden 2023 Loans', 'Sanoun Transactions 2024', 'Sanoun Transactions 2025', '2024 Shea butter shipping', 'Yiriden mileages', 'Truck Revenue Projections', 'Yiriden 2022', 'Real Estate - Horton Rd']

Loading sheet: 'Yiriden 2023 Loans'

Loaded data with shape: (121, 14)
Columns: ['Lender', 'Amount', 'Term', 'Rate', 'Payment', 'Origination Date', 'Truck', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']

First 5 rows:


,Lender,Amount,Term,Rate,Payment,Origination Date,Truck,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,BHG,89395.0,120.0,0.1824,1624.60,2023-04-18,Actros 1,NaN,Payment #,Date,Amount,Interest,Principal,Balance
1,Fidelity 401(k) loan,36000.0,59.0,0.0925,350.63,2023-07-16,Actros 2,NaN,1,2023-05-20 00:00:00,1624.6,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,2,2023-06-20 00:00:00,1624.6,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,3,2023-07-20 00:00:00,1624.6,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,4,2023-08-20 00:00:00,1624.6,NaN,NaN,NaN



Data types:
Lender                      object
Amount                     float64
Term                       float64
Rate                       float64
Payment                    float64
Origination Date    datetime64[ns]
Truck                       object
Unnamed: 7                 float64
Unnamed: 8                  object
Unnamed: 9                  object
Unnamed: 10                 object
Unnamed: 11                 object
Unnamed: 12                 object
Unnamed: 13                 object
dtype: object


In [2]:
# Query interface for formula dependency analysis
# The pipeline has already analyzed all formulas and cached the results

import pickle
from pathlib import Path
from spreadsheet_analyzer.graph_db.query_interface import create_enhanced_query_interface

# Load cached formula analysis
cache_file = Path(r".pipeline_cache/Business Accounting_formula_analysis.pkl")
with open(cache_file, 'rb') as f:
    formula_analysis = pickle.load(f)

query_interface = create_enhanced_query_interface(formula_analysis)

# Convenience functions for graph queries
def get_cell_dependencies(sheet, cell_ref):
    """Get complete dependency information for a specific cell."""
    result = query_interface.get_cell_dependencies(sheet, cell_ref)
    print(f"\nCell {sheet}!{cell_ref}:")
    print(f"  Has formula: {result.has_formula}")
    if result.formula:
        print(f"  Formula: {result.formula}")
    if result.direct_dependencies:
        print(f"  Direct dependencies: {', '.join(result.direct_dependencies[:5])}")
        if len(result.direct_dependencies) > 5:
            print(f"    ...and {len(result.direct_dependencies) - 5} more")
    if result.direct_dependents:
        print(f"  Cells that depend on this: {', '.join(result.direct_dependents[:5])}")
        if len(result.direct_dependents) > 5:
            print(f"    ...and {len(result.direct_dependents) - 5} more")
    return result

def find_cells_affecting_range(sheet, start_cell, end_cell):
    """Find all cells that affect any cell within the specified range."""
    result = query_interface.find_cells_affecting_range(sheet, start_cell, end_cell)
    print(f"\nCells affecting range {sheet}!{start_cell}:{end_cell}:")
    for cell, deps in list(result.items())[:5]:
        print(f"  {cell} depends on: {', '.join(deps[:3])}")
        if len(deps) > 3:
            print(f"    ...and {len(deps) - 3} more")
    if len(result) > 5:
        print(f"  ...and {len(result) - 5} more cells")
    return result

def get_formula_statistics():
    """Get comprehensive statistics about formulas in the workbook."""
    stats = query_interface.get_formula_statistics_with_ranges()
    print("\nFormula Statistics:")
    print(f"  Total formulas: {stats['total_formulas']:,}")
    print(f"  Formulas with dependencies: {stats['formulas_with_dependencies']:,}")
    print(f"  Unique cells referenced: {stats['unique_cells_referenced']:,}")
    print(f"  Max dependency depth: {stats['max_dependency_depth']} levels")
    print(f"  Circular references: {stats['circular_reference_chains']}")
    print(f"  Formula complexity score: {stats['complexity_score']}/100")
    return stats

def find_empty_cells_in_formula_ranges(sheet):
    """Find empty cells that are part of formula ranges."""
    result = query_interface.find_empty_cells_in_formula_ranges(sheet)
    print(f"\nEmpty cells in formula ranges for sheet '{sheet}':")
    if result:
        print(f"  Found {len(result)} empty cells")
        # Group by rows for display
        rows = {}
        for cell in list(result)[:20]:
            row_num = ''.join(filter(str.isdigit, cell))
            if row_num not in rows:
                rows[row_num] = []
            rows[row_num].append(cell)
        for row, cells in list(rows.items())[:5]:
            print(f"  Row {row}: {', '.join(cells)}")
        if len(result) > 20:
            print(f"  ...and {len(result) - 20} more")
    else:
        print("  No empty cells found in formula ranges")
    return result


## 🔍 Formula Analysis Tools

You have TWO approaches available for formula analysis:

### 1️⃣ Graph-Based Dependency Analysis (Recommended for Complex Files)
The deterministic pipeline has analyzed all formulas and created a dependency graph. These tools are robust and handle complex Excel files:

- **get_cell_dependencies** - Analyze what a cell depends on and what depends on it
- **find_cells_affecting_range** - Find all cells that affect a specific range
- **find_empty_cells_in_formula_ranges** - Find gaps in data that formulas reference
- **get_formula_statistics** - Get overall statistics about formulas
- **find_circular_references** - Find all circular reference chains

### 2️⃣ Formulas Library for Advanced Formula Evaluation (Recommended)
Robust formula evaluation using the 'formulas' library that handles complex Excel files:

- **load_excel_with_formulas** - Load Excel file for formula evaluation
- **evaluate_cell** - Get calculated cell values and formulas
- **set_cell_and_recalculate** - What-if analysis with recalculation
- **get_cell_dependencies_formulas** - Track formula dependencies
- **export_formulas_model** - Export model to JSON
- **get_formulas_help** - Get detailed help

✅ **Recommended**: The formulas library handles complex Excel files much better than other alternatives.

### Usage:
All tools are available through the tool-calling interface. Use graph-based analysis for quick dependency queries, and the formulas library for accurate formula evaluation and what-if analysis.


## --- LLM Analysis Start ---